In [1]:
import cv2
import mediapipe as mp
import numpy as np

# Setup Mediapipe
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=1, min_detection_confidence=0.7)
mp_draw = mp.solutions.drawing_utils

def get_angle(points):
    """Calculates angle between three points (mcp, pip, dip)"""
    a, b, c = np.array(points[0]), np.array(points[1]), np.array(points[2])
    rad = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(rad * 180.0 / np.pi)
    return 360-angle if angle > 180.0 else angle

cap = cv2.VideoCapture(1)

while cap.isOpened():
    success, img = cap.read()
    if not success: break
    
    img = cv2.flip(img, 1)
    results = hands.process(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    
    if results.multi_hand_landmarks:
        for handLms in results.multi_hand_landmarks:
            lm = handLms.landmark
            
            # Get Index Angle (5,6,7) and Middle Angle (9,10,11)
            idx_angle = get_angle([[lm[5].x, lm[5].y], [lm[6].x, lm[6].y], [lm[7].x, lm[7].y]])
            mid_angle = get_angle([[lm[9].x, lm[9].y], [lm[10].x, lm[10].y], [lm[11].x, lm[11].y]])
            
            # Logic for Letter 'L': Index is straight, Middle is folded
            is_l_shape = idx_angle > 160 and mid_angle < 100
            
            color = (0, 255, 0) if is_l_shape else (0, 0, 255)
            label = "L - CORRECT" if is_l_shape else "SEARCHING..."
            
            mp_draw.draw_landmarks(img, handLms, mp_hands.HAND_CONNECTIONS)
            cv2.putText(img, label, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 3)

    cv2.imshow("ASL Learning Tool", img)
    if cv2.waitKey(1) & 0xFF == ord('q'): break

cap.release()
cv2.destroyAllWindows()

In [4]:
import mediapipe as mp
print(mp)
print(mp.__file__)


<module 'mediapipe' from 'D:\\HCI project\\mp_env\\lib\\site-packages\\mediapipe\\__init__.py'>
D:\HCI project\mp_env\lib\site-packages\mediapipe\__init__.py


In [3]:
import sys
print(sys.version)


3.10.11 (tags/v3.10.11:7d4cc5a, Apr  5 2023, 00:38:17) [MSC v.1929 64 bit (AMD64)]


In [2]:
import mediapipe as mp
print(mp.__file__)
print(mp.solutions)

D:\HCI project\mp_env\lib\site-packages\mediapipe\__init__.py
<module 'mediapipe.python.solutions' from 'D:\\HCI project\\mp_env\\lib\\site-packages\\mediapipe\\python\\solutions\\__init__.py'>
